Cargamos las resoluciones en una db por cuestiones de eficiencia y performance a la hora de hacer el procesamiento posterior.
Se almacena el nombre del archivo, tipo, contenido, fecha de almacenamiento, vigencia. Por el momento la vigencia no se utiliza, pero eventualmente servirá para filtrar las normativas que ya no están vigentes.

In [ ]:
!pip install PyPDF2
!pip install pdf2image
!pip install pytesseract

In [ ]:
!apt-get update
!apt-get install -y poppler-utils
!apt-get install -y tesseract-ocr tesseract-ocr-spa

In [ ]:
import os
import sqlite3
from datetime import datetime
from PyPDF2 import PdfReader, errors as pdf_errors
from pdf2image import convert_from_path
import pytesseract

Ver de modularizar

In [ ]:
# Configurar rutas de Poppler y Tesseract
ruta_poppler = "/usr/bin"  # Cambia si usas otro sistema o configuración
pytesseract.pytesseract.tesseract_cmd = "/usr/bin/tesseract" 

# Crear/conectar a la base de datos
db_path = "/content/normativa_dgcye.db"
conn = sqlite3.connect(db_path)
cursor = conn.cursor()

# Crear la tabla si no existe
cursor.execute('''
    CREATE TABLE IF NOT EXISTS archivos (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        nombre_archivo TEXT,
        tipo_archivo TEXT,
        contenido TEXT,
        fecha_procesamiento TEXT,
        vigente BOOLEAN
    )
''')

def archivo_existe(nombre_archivo):
    cursor.execute('SELECT 1 FROM archivos WHERE nombre_archivo = ?', (nombre_archivo,))
    return cursor.fetchone() is not None  # Devuelve True si existe

# Función para insertar resoluciones en la base de datos si no existe previamente
def cargar_resoluciones(nombre_archivo, tipo_archivo, contenido):
    if archivo_existe(nombre_archivo):
        print(f"El archivo '{nombre_archivo}' ya existe en la base de datos. No se agregará.")
        return

    fecha = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    cursor.execute('''
        INSERT INTO archivos (nombre_archivo, tipo_archivo, contenido, fecha_procesamiento, vigente)
        VALUES (?, ?, ?, ?, ?)
    ''', (nombre_archivo, tipo_archivo, contenido, fecha, True))
    conn.commit()
    print(f"Archivo '{nombre_archivo}' agregado correctamente.")

# Función para extraer texto de PDFs normales
def extraer_texto_pdf(ruta_archivo):
    texto = ""
    try:
        with open(ruta_archivo, 'rb') as archivo_pdf:
            lector_pdf = PdfReader(archivo_pdf)
            for pagina in lector_pdf.pages:
                texto += pagina.extract_text() or ""
    except pdf_errors.PdfReadError as e:
        print(f"Error al leer el archivo {ruta_archivo}: {e}")
    except Exception as e:
        print(f"Ocurrió un error al procesar {ruta_archivo}: {e}")
    return texto

# Función para extraer texto usando OCR en PDFs con imágenes
def extraer_texto_ocr(ruta_archivo):
    texto = ""
    try:
        paginas_imagen = convert_from_path(ruta_archivo, poppler_path=ruta_poppler)
        for pagina_imagen in paginas_imagen:
            texto += pytesseract.image_to_string(pagina_imagen, lang='spa')
    except Exception as e:
        print(f"Error al aplicar OCR a {ruta_archivo}: {e}")
    return texto

# Función principal para procesar PDFs
def procesar_pdf(ruta_archivo):
    texto = extraer_texto_pdf(ruta_archivo)
    tipo_archivo = "PDF"
    if not texto.strip():
        print(f"Aplicando OCR al archivo {ruta_archivo}")
        texto = extraer_texto_ocr(ruta_archivo)
        tipo_archivo = "OCR"
    return texto, tipo_archivo

# Directorio con los PDFs (modifica si es necesario)
ruta_directorio = "/content/resoluciones/"
archivos_pdf = [f for f in os.listdir(ruta_directorio) if f.endswith('.pdf')]

# Procesar y cargar los PDFs en la base de datos
for archivo in archivos_pdf:
    ruta_completa = os.path.join(ruta_directorio, archivo)
    texto, tipo_archivo = procesar_pdf(ruta_completa)
    if texto:
        cargar_resoluciones(archivo, tipo_archivo, texto)

# Verificar el contenido de la base de datos
cursor.execute('SELECT * FROM archivos')
print(cursor.fetchall())

# Cerrar conexión a la base
conn.close()

Esta celda permite ver el contenido de una fila a elección

In [ ]:
# Conectar a la base de datos SQLite
conn = sqlite3.connect("/content/normativa_dgcye.db")
cursor = conn.cursor()

# Consultar una línea específica (por ID o algún otro criterio)
id_linea = 1  # Cambia este valor al ID que quieras consultar

cursor.execute('SELECT * FROM archivos WHERE id = ?', (id_linea,))
linea = cursor.fetchone()

# Verificar si se encontró la línea y mostrar el contenido
if linea:
    print(f"\nID: {linea[0]}")
    print(f"Nombre del archivo: {linea[1]}")
    print(f"Tipo de archivo: {linea[2]}")
    print(f"Contenido:\n{linea[3]}")
    print(f"Fecha de procesamiento: {linea[4]}")
    print(f"Vigente: {linea[5]}")
else:
    print(f"No se encontró ninguna línea con ID = {id_linea}")

# Cerrar la conexión a la base de datos
conn.close()